# Prepare data for Pipeline

## Setup and settings

In [1]:
# Importing packages
import os
import yaml
import logging
import pandas as pd
from Bio.Seq import Seq
from qiime2 import Artifact, Visualization, Metadata
from qiime2.plugins import demux
from qiime2.plugins.cutadapt.methods import trim_paired, trim_single
from qiime2.plugins.demux.methods import filter_samples

### Receiving the parameters

The following cell can receive parameters using the [papermill](https://papermill.readthedocs.io/en/latest/) tool.

In [2]:
params_path = os.path.join('..', 'params', 'parameter-set-01.yaml')
experiment_name = 'exp-01'
base_dir = os.path.join('/', 'home', 'username', 'pipeline-dir')
manifest_file = os.path.join(base_dir, 'data', 'raw', 'manifest', 'manifest.csv')
replace_files = False
trim = None
metadata_file = None
threads = 1

In [3]:
# Parameters
experiment_name = "thayane-PM-paired-end-trim"
base_dir = "/home/lauro/nupeb/rede-micro/redemicro-thayane"
manifest_file = (
    "/home/lauro/nupeb/rede-micro/redemicro-thayane/data/manifest-paired.csv"
)
metadata_file = (
    "/home/lauro/nupeb/rede-micro/redemicro-thayane/data/paired-end-metadata.tsv"
)
class_col = "above_10"
classifier_file = (
    "/home/lauro/nupeb/16S_classifiers_qiime2/silva-138-99-nb-classifier.qza"
)
replace_files = False
phred = 20
trunc_f = 0
trunc_r = 0
overlap = 12
threads = 6
trim = {
    "overlap": 8,
    "forward_primer": "CCTACGGGRSGCAGCAG",
    "reverse_primer": "GGACTACHVGGGTWTCTAAT",
}


### Defining names and paths

In [4]:
# Define the output folder path
out_dir = os.path.join(base_dir, 'experiments', experiment_name, 'qiime-artifacts')
img_folder = os.path.abspath(os.path.join(base_dir, 'experiments', experiment_name, 'imgs'))

# Create path if it not exists
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)
    print(f'New qiime-artifacts folder path created: {out_dir}')

# Create path if it not exists
if not os.path.isdir(img_folder):
    os.makedirs(img_folder)
    print(f'New img folder path created: {img_folder}')

# Define the output artifact full path
demux_file = os.path.join(out_dir, 'demux-paired.qza')
demux_view = os.path.join(out_dir, 'demux-paired.qzv')
demux_file_trim = os.path.join(out_dir, 'demux-paired-trim.qza')
demux_view_trim = os.path.join(out_dir, 'demux-paired-trim.qzv')

New qiime-artifacts folder path created: /home/lauro/nupeb/rede-micro/redemicro-thayane/experiments/thayane-PM-paired-end-trim/qiime-artifacts
New img folder path created: /home/lauro/nupeb/rede-micro/redemicro-thayane/experiments/thayane-PM-paired-end-trim/imgs


## Step execution

This Step import all `fastq` files in a **QIIME2 Artifact** object and save it to a new `qza` file of `SampleData[PairedEndSequencesWithQuality]`

In [5]:
manifest_df = pd.read_csv(manifest_file)
n_directions = len(manifest_df['direction'].unique())
if n_directions == 1:
    d_type = 'SampleData[SequencesWithQuality]'
    v_type = 'SingleEndFastqManifestPhred33'
elif n_directions == 2:
    d_type = 'SampleData[PairedEndSequencesWithQuality]'
    v_type = 'PairedEndFastqManifestPhred33'
else:
    print(f'ERROR: invalid number of directions {n_directions}')

In [6]:
# If the metadata file is not defined, use the default metadata file
metadata_qa = Metadata.load(metadata_file)

In [7]:
# If the demux file does not exist or if the old file will be replaced
if not os.path.isfile(demux_file) or replace_files:
    
    # Import data and create an artifact object
    artifact = Artifact.import_data(
        d_type, 
        manifest_file, 
        view_type=v_type)

    artifact = filter_samples(demux=artifact, metadata=metadata_qa).filtered_demux
    # Save the artifact object to a new qza file
    artifact.save(demux_file)

else:
    artifact = Artifact.load(demux_file)

In [8]:
if not os.path.isfile(demux_view) or replace_files: 
    # Generate e visualization of the Artifact
    demux_summary = demux.visualizers.summarize(artifact)

    # Save a new visualization file based on the qza file
    Visualization.save(demux_summary.visualization, filepath=demux_view)
    
    demux_view_obj = demux_summary.visualization
else:
    demux_view_obj = Visualization.load(demux_view)

## Step report


In [9]:
print(demux_view_obj)

<visualization: Visualization uuid: 4615203b-c3dd-420f-ad7d-77545c5cd848>


In [10]:
# Render Visualization
demux_view_obj

<visualization: Visualization uuid: 4615203b-c3dd-420f-ad7d-77545c5cd848>

[cutadapt](https://docs.qiime2.org/2022.2/plugins/available/cutadapt/trim-paired/)

We trim the forward primer and the reverse complement of the reverse primer from the forward reads. We trim the reverse primer and reverse complement of the forward primer from the reverse reads.

In [11]:
demux_file_trim = os.path.join(out_dir, 'demux-paired-trim.qza')
demux_view_trim = os.path.join(out_dir, 'demux-paired-trim.qzv')

if trim and (not os.path.isfile(demux_file_trim) or replace_files):
    forward_primer = [trim['forward_primer']] # ['CCTACGGGRSGCAGCAG']
    reverse_primer = [trim['reverse_primer']] # ['GGACTACHVGGGTWTCTAAT']
    forward_reverse_complement = [str(Seq(forward_primer[0]).reverse_complement())]
    reverse_reverse_complement = [str(Seq(reverse_primer[0]).reverse_complement())]

    res = None
    
    if n_directions == 1:
        res = trim_single(
            demultiplexed_sequences=artifact,
            front=forward_primer,
            adapter=reverse_reverse_complement,
            cores=threads,
            indels=False,
            match_read_wildcards=True,
            match_adapter_wildcards=True,
            error_rate=0.01,
            discard_untrimmed=True,
        ).trimmed_sequences
    elif n_directions == 2:
        res = trim_paired(
            demultiplexed_sequences=artifact,
            front_f=forward_primer,
            front_r=reverse_primer,
            # adapter_f=reverse_reverse_complement,
            # adapter_r=forward_reverse_complement,
            cores=threads,
            overlap=trim['overlap'],
            indels=False,
            match_read_wildcards=True,
            match_adapter_wildcards=True,
            error_rate=0.01,
            discard_untrimmed=True,
        ).trimmed_sequences
    else:
        print(f'ERROR: invalid number of directions {n_directions}')

    if res is not None:
        res.save(demux_file_trim)
        Visualization.save(demux.visualizers.summarize(res).visualization, filepath=demux_view_trim)

Running external command line application. This may print messages to stdout and/or stderr.
The commands to be run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: cutadapt --cores 6 --error-rate 0.01 --times 1 --overlap 8 --minimum-length 1 -o /tmp/q2-CasavaOneEightSingleLanePerSampleDirFmt-jw2nnpz9/M05_0_L001_R1_001.fastq.gz -p /tmp/q2-CasavaOneEightSingleLanePerSampleDirFmt-jw2nnpz9/M05_1_L001_R2_001.fastq.gz --front CCTACGGGRSGCAGCAG -G GGACTACHVGGGTWTCTAAT --no-indels --match-read-wildcards --discard-untrimmed /tmp/qiime2-archive-zjnprj4v/93655b56-b0ce-4fe7-beb6-76f378ec4020/data/M05_0_L001_R1_001.fastq.gz /tmp/qiime2-archive-zjnprj4v/93655b56-b0ce-4fe7-beb6-76f378ec4020/data/M05_1_L001_R2_001.fastq.gz



This is cutadapt 3.5 with Python 3.8.12
Command line parameters: --cores 6 --error-rate 0.01 --times 1 --overlap 8 --minimum-length 1 -o /tmp/q2-CasavaOneEightSingleLanePerSampleDirFmt-jw2nnpz9/M05_0_L001_R1_001.fastq.gz -p /tmp/q2-CasavaOneEightSingleLanePerSampleDirFmt-jw2nnpz9/M05_1_L001_R2_001.fastq.gz --front CCTACGGGRSGCAGCAG -G GGACTACHVGGGTWTCTAAT --no-indels --match-read-wildcards --discard-untrimmed /tmp/qiime2-archive-zjnprj4v/93655b56-b0ce-4fe7-beb6-76f378ec4020/data/M05_0_L001_R1_001.fastq.gz /tmp/qiime2-archive-zjnprj4v/93655b56-b0ce-4fe7-beb6-76f378ec4020/data/M05_1_L001_R2_001.fastq.gz
Processing reads on 6 cores in paired-end mode ...


Finished in 0.36 s (57 µs/read; 1.04 M reads/minute).

=== Summary ===

Total read pairs processed:              6,307
  Read 1 with adapter:                   6,057 (96.0%)
  Read 2 with adapter:                   5,115 (81.1%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:            1,379 (21.9%)
Pairs written (passing filters):         4,928 (78.1%)

Total basepairs processed:     3,216,570 bp
  Read 1:     1,923,635 bp
  Read 2:     1,292,935 bp
Total written (filtered):      2,297,458 bp (71.4%)
  Read 1:     1,402,262 bp
  Read 2:       895,196 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 6057 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
8	1	0.1	0	1
14	2	0.0	0	2
15	1	0.0	0	1
17	3	0.0	0	3
18	916	0.0	0	916
19	1016	0.0	0	1016
20	1108	0.0	0	1108
21	1053	0.0	0	1053
22	1099	0.0	0

Finished in 0.13 s (141 µs/read; 0.43 M reads/minute).

=== Summary ===

Total read pairs processed:                936
  Read 1 with adapter:                     888 (94.9%)
  Read 2 with adapter:                     781 (83.4%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:              196 (20.9%)
Pairs written (passing filters):           740 (79.1%)

Total basepairs processed:       477,360 bp
  Read 1:       285,480 bp
  Read 2:       191,880 bp
Total written (filtered):        344,920 bp (72.3%)
  Read 1:       210,509 bp
  Read 2:       134,411 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 888 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
17	1	0.0	0	1
18	127	0.0	0	127
19	172	0.0	0	172
20	155	0.0	0	155
21	150	0.0	0	150
22	158	0.0	0	158
23	124	0.0	0	124
46	1	0.0	0	1


=== Seco

Finished in 0.57 s (16 µs/read; 3.74 M reads/minute).

=== Summary ===

Total read pairs processed:             35,289
  Read 1 with adapter:                  34,139 (96.7%)
  Read 2 with adapter:                  32,305 (91.5%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:            4,014 (11.4%)
Pairs written (passing filters):        31,275 (88.6%)

Total basepairs processed:    17,997,390 bp
  Read 1:    10,763,145 bp
  Read 2:     7,234,245 bp
Total written (filtered):     14,579,866 bp (81.0%)
  Read 1:     8,898,603 bp
  Read 2:     5,681,263 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 34139 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
10	2	0.0	0	2
13	1	0.0	0	1
14	1	0.0	0	1
15	1	0.0	0	1
16	3	0.0	0	3
17	12	0.0	0	12
18	5146	0.0	0	5146
19	6114	0.0	0	6114
20	6307	0.0	0	6307


...

Finished in 2.41 s (10 µs/read; 6.28 M reads/minute).

=== Summary ===

Total read pairs processed:            251,880
  Read 1 with adapter:                 243,664 (96.7%)
  Read 2 with adapter:                 230,412 (91.5%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:           28,790 (11.4%)
Pairs written (passing filters):       223,090 (88.6%)

Total basepairs processed:   128,458,800 bp
  Read 1:    76,823,400 bp
  Read 2:    51,635,400 bp
Total written (filtered):    104,001,955 bp (81.0%)
  Read 1:    63,477,240 bp
  Read 2:    40,524,715 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 243664 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
8	1	3.8	0	1
9	1	1.0	0	1
10	6	0.2	0	6
11	3	0.1	0	3
12	4	0.0	0	4
13	2	0.0	0	2
14	10	0.0	0	10
15	4	0.0	0	4
16	8	0.0	0	8
17	66	0.0	0	66
18	3

Finished in 2.42 s (10 µs/read; 6.04 M reads/minute).

=== Summary ===

Total read pairs processed:            243,421
  Read 1 with adapter:                 234,893 (96.5%)
  Read 2 with adapter:                 221,420 (91.0%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:           29,614 (12.2%)
Pairs written (passing filters):       213,807 (87.8%)

Total basepairs processed:   124,144,710 bp
  Read 1:    74,243,405 bp
  Read 2:    49,901,305 bp
Total written (filtered):     99,673,922 bp (80.3%)
  Read 1:    60,836,006 bp
  Read 2:    38,837,916 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 234893 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
8	3	3.7	0	3
9	1	0.9	0	1
10	4	0.2	0	4
11	7	0.1	0	7
12	4	0.0	0	4
13	2	0.0	0	2
14	10	0.0	0	10
15	12	0.0	0	12
16	12	0.0	0	12
17	56	0.0	0	56


Finished in 4.04 s (10 µs/read; 6.16 M reads/minute).

=== Summary ===

Total read pairs processed:            414,147
  Read 1 with adapter:                 400,620 (96.7%)
  Read 2 with adapter:                 378,571 (91.4%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:           47,620 (11.5%)
Pairs written (passing filters):       366,527 (88.5%)

Total basepairs processed:   211,214,970 bp
  Read 1:   126,314,835 bp
  Read 2:    84,900,135 bp
Total written (filtered):    170,876,279 bp (80.9%)
  Read 1:   104,295,532 bp
  Read 2:    66,580,747 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 400620 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
8	4	6.3	0	4
9	5	1.6	0	5
10	4	0.4	0	4
11	5	0.1	0	5
12	3	0.0	0	3
13	4	0.0	0	4
14	16	0.0	0	16
15	17	0.0	0	17
16	27	0.0	0	27
17	104	0.0	0	10

Finished in 2.06 s (10 µs/read; 5.93 M reads/minute).

=== Summary ===

Total read pairs processed:            204,057
  Read 1 with adapter:                 197,100 (96.6%)
  Read 2 with adapter:                 186,765 (91.5%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:           23,508 (11.5%)
Pairs written (passing filters):       180,549 (88.5%)

Total basepairs processed:   104,069,070 bp
  Read 1:    62,237,385 bp
  Read 2:    41,831,685 bp
Total written (filtered):     84,169,976 bp (80.9%)
  Read 1:    51,372,099 bp
  Read 2:    32,797,877 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 197100 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
8	3	3.1	0	3
9	1	0.8	0	1
10	1	0.2	0	1
11	4	0.0	0	4
13	1	0.0	0	1
14	10	0.0	0	10
15	5	0.0	0	5
16	6	0.0	0	6
17	47	0.0	0	47
18	29818	0.0	0	29

Finished in 1.13 s (12 µs/read; 5.15 M reads/minute).

=== Summary ===

Total read pairs processed:             96,698
  Read 1 with adapter:                  93,525 (96.7%)
  Read 2 with adapter:                  88,471 (91.5%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:           11,087 (11.5%)
Pairs written (passing filters):        85,611 (88.5%)

Total basepairs processed:    49,315,980 bp
  Read 1:    29,492,890 bp
  Read 2:    19,823,090 bp
Total written (filtered):     39,911,283 bp (80.9%)
  Read 1:    24,359,930 bp
  Read 2:    15,551,353 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 93525 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
8	2	1.5	0	2
10	2	0.1	0	2
11	2	0.0	0	2
12	1	0.0	0	1
13	1	0.0	0	1
14	1	0.0	0	1
15	4	0.0	0	4
16	1	0.0	0	1
17	29	0.0	0	29
18	14577	0.0	0	1457

In [12]:
demux.visualizers.summarize(res).visualization

<visualization: Visualization uuid: b1763ae8-71b8-4ca8-87d8-7784bf918167>